<a href="https://www.kaggle.com/code/maita27/titanic?scriptVersionId=247946840" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
train_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

y_train = train_data['Survived']
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']

x_train = pd.get_dummies(train_data[features])
x_test = pd.get_dummies(test_data[features])

#bitchass feature engineering and preproc
x_train['FamSize'] = train_data['SibSp'] + train_data['Parch']
x_train['isAlone'] = x_train['FamSize'] == 0
x_train['Age'] = x_train['Age'].fillna(x_train['Age'].median())
x_train['Age'] = scaler.fit_transform(x_train[['Age']])

x_test['FamSize'] = test_data['SibSp'] + test_data['Parch']
x_test['isAlone'] = x_test['FamSize'] == 0
x_test['Age'] = x_test['Age'].fillna(x_test['Age'].median())
x_test['Age'] = scaler.fit_transform(x_test[['Age']])

x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

x_train.head()

,Pclass,Age,SibSp,Parch,Sex_female,Sex_male,FamSize,isAlone
0,3,-0.565736,1,0,False,True,1,False
1,1,0.663861,1,0,True,False,1,False
2,3,-0.258337,0,0,True,False,0,True
3,1,0.433312,1,0,True,False,1,False
4,3,0.433312,0,0,False,True,0,True


In [5]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

base_learners = [
    ('rf', RandomForestClassifier(n_estimators = 100, random_state = 42)),
    ('gb', GradientBoostingClassifier(n_estimators = 100, random_state = 42)),
    ('svc', SVC(probability = True, kernel = 'rbf', C = 1, gamma = 'auto'))
]

meta_learner = LogisticRegression()

model = StackingClassifier(
    estimators = base_learners,
    final_estimator = meta_learner,
    passthrough = True, #original features in metalearner
    cv = 5
)

model.fit(x_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('rf', RandomForestClassifier(random_state=42)),
                               ('gb',
                                GradientBoostingClassifier(random_state=42)),
                               ('svc',
                                SVC(C=1, gamma='auto', probability=True))],
                   final_estimator=LogisticRegression(), passthrough=True)

In [6]:
predictions = model.predict(x_test)

In [7]:
output = pd.DataFrame({
    'PassengerId' : test_data['PassengerId'],
    'Survived' : predictions
})
output.head()
output.to_csv('submission.csv', index = False)